In [1]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Step 2: Load Dataset
df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']

# Step 3: Label Encoding
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Step 4: Text Cleaning Function
def clean_text(text):
    text = text.lower()
    text = ''.join(c for c in text if c not in string.punctuation)
    words = text.split()
    words = [ps.stem(w) for w in words if w not in stopwords.words('english')]
    return ' '.join(words)

df['cleaned'] = df['message'].apply(clean_text)

# Step 5: Vectorization
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['cleaned']).toarray()
y = df['label'].values

# Step 6: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train Models
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

model_et = ExtraTreesClassifier()
model_et.fit(X_train, y_train)

# Step 8: Evaluate
for model, name in zip([model_nb, model_et], ["Naive Bayes", "Extra Trees"]):
    y_pred = model.predict(X_test)
    print(f"\n📊 Model: {name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

# Step 9: Save Best Model (Naive Bayes)
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump((model_nb, tfidf), f)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SALAAR\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.



📊 Model: Naive Bayes
Accuracy: 0.97847533632287
[[965   0]
 [ 24 126]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       1.00      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115


📊 Model: Extra Trees
Accuracy: 0.979372197309417
[[963   2]
 [ 21 129]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.98      0.86      0.92       150

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

